# Mock Belt Exam - Data Enrichment


- Blank Notebook for Lecture
- 08/04/22
- Purvi Kansara

## Instructions

Data Enrichment Mock Exam

API results:

https://drive.google.com/file/d/10iWPhZtId0R9RCiVculSozCwldG-V3eH/view?usp=sharing

1. Read in the json file
2. Separate the records into 4 tables each a pandas dataframe
3. Transform
    In this case remove dollar signs from funded amount in the financials records and convert to numeric datatype
4. Create a database with SQLAlchemy and add the tables to the datbase
5. Perform a hypothesis test to determine if there is a signficant difference between the funded amount when it is all males and when there is at least one female in the group.

# ETL of JSON File

## Extract

In [ ]:
## Loading json file


In [ ]:
## explore each key 


In [ ]:
## display meta


In [ ]:
## display data


In [ ]:
## preview the dictionary


In [ ]:
## preview just the keys


In [ ]:
## what does the crowd key look like?


In [ ]:
## checking single entry of crowd


In [ ]:
## making crowd a dataframe


In [ ]:
## making demographics a dataframe


In [ ]:
## making financials a dataframe


In [ ]:
## making use a dataframe


## Transform

In [ ]:
## fixing funded amount column


## Load

In [ ]:
## loading mysql credentials


In [ ]:
## creating connection to database with sqlalchemy


In [ ]:
## Check if database exists, if not, create it


In [ ]:
## saving dataframes to database


In [ ]:
## checking if tables created


# Hypothesis Testing

### State the Hypothesis & Null Hypothesis 

- $H_0$ (Null Hypothesis): Funded amount is the same for teams that contain at least 1 female and teams that are all male.
- $H_A$ (Alternative Hypothesis): There is a significant difference between the funded amount for teams that contain at least 1 female and teams that are all male.

- Based upon the [Choosing the Right Hypothesis Test workflow from  the LP]( https://login.codingdojo.com/m/376/12533/88117):
    - **The appropriate test to perform would be:**
        - Since we are measuring a numeric quantity (funded amount)
        - and we are comparing 2 groups/samples. 
        - We therefore want to perform a 2-sample t-test, A.K.A. an independent t-test.
    
- **According the the work flow, the 2-sample T-Test has the following assumptions:** 
    - No significant outliers
    - Normality
    - Equal Variance

## Getting the Group Data 

- The next step is to get the data for each group in separate variables. All of the approaches below will lead to the same result: a `male_df` and `female_df` variable.

### Approach 1: Using the MySQL Database to Get DF to Filter

In [ ]:
q = """SELECT 
    f.id, f.funded_amount, d.borrower_genders
FROM
    financials AS f
        JOIN
    demographics AS d ON f.id = d.id;"""
df = pd.read_sql(q,engine)
df

In [ ]:
## Create a column that defines the 2 groups, has female or not
df['has_female'] = df['borrower_genders'].str.contains('female', case=False)
df

In [ ]:
## Separate the column of interest based on the groups
male_df = df.loc[ df['has_female']==False, ['funded_amount','has_female']]
female_df = df.loc[ df['has_female']==True, ['funded_amount','has_female']]
print(f"There are {len(female_df)} campaigns that had females on the team." )
print(f"There are {len(male_df)} campaigns that only had males on the team." )

### Approach 2: Using the MySQL database to make the male_df and female_df

>- Due to a quirk with using "%" with sqlalchemy queries, in order to use a LIKE command with "%" for "%female":
    1. Add quotation marks around the "%" expression. 
    ```python
    q = '''SELECT 
    f.funded_amount, d.borrower_genders
FROM
    financials AS f
        JOIN
    demographics AS d ON f.id = d.id
WHERE
    d.borrower_genders LIKE "%female%";'''
    ```
    2. Use the sqlalchemy text function when running your query. 
    ```python
    from sqlalchemy import text
    female_df = pd.read_sql(text(q),engine)
    ```

In [ ]:
## importing text function to use on query with a "%" in it
from sqlalchemy import text

In [ ]:
## query to get campaigns that included female borrowers
q = '''SELECT 
    f.funded_amount,  
    d.borrower_genders LIKE "%female%" as "has_female"
FROM
    financials AS f
        JOIN
    demographics AS d ON f.id = d.id
WHERE
    d.borrower_genders LIKE "%female%";'''
female_df = pd.read_sql(text(q),engine)
female_df

In [ ]:
## query to get campaigns that were only male borrowers
q = """SELECT 
    f.funded_amount, 
    d.borrower_genders LIKE "%female%" as "has_female"

FROM
    financials AS f
        JOIN
    demographics AS d ON f.id = d.id
WHERE
    d.borrower_genders NOT LIKE '%female%';"""
male_df = pd.read_sql(text(q),engine)
male_df

In [ ]:
print(f"There are {len(female_df)} campaigns that had females on the team." )
print(f"There are {len(male_df)} campaigns that only had males on the team." )

### Approach 3: Use pd.merge to join the DataFrames

In [ ]:
df = pd.merge(financials, demo, on='id')
df

In [ ]:
df['has_female'] = df['borrower_genders'].str.contains('female', case=False)
df

In [ ]:
## Separate the column of interest based on the groups
male_df = df.loc[ df['has_female']==False, ['funded_amount','has_female']]
female_df = df.loc[ df['has_female']==True,['funded_amount','has_female']]

print(f"There are {len(female_df)} campaigns that had females on the team." )
print(f"There are {len(male_df)} campaigns that only had males on the team." )

## Visualize Group Means

In [ ]:
## concatenate the two dataframes for visualziation.
plot_df = pd.concat([male_df, female_df], axis=0)
plot_df

In [ ]:
## visualizing means. ci=68 makes easier to compare error bars (will discuss in class)
ax = sns.barplot(data=plot_df, x='has_female', y='funded_amount', ci=68)

In [ ]:
## now that we have visualized the groups, we can save a final male_group and female_group
# that are a pandas Series. This will make the rest of our workflow simpler than if 
# we still had a dataframe

female_group = female_df['funded_amount']
male_group = male_df['funded_amount']
display(female_group.head(), male_group.head())

## Checking Assumptions of 2-Sample T-test

- **According the the work flow, the 2-sample T-Test has the following assumptions:** 
    - No significant outliers
    - Normality
    - Equal Variance

### Checking for Outliers

- Check each group SEPARATELY!

In [ ]:
## Checking for abs vlaue of z-scores that are > 3
is_outlier_females = np.abs(stats.zscore(female_group)) > 3
print(f"There are {is_outlier_females.sum()} outliers in the female group out of {len(female_group)})")

In [ ]:
female_df.loc[~is_outlier_females]

In [ ]:
## removing outliers from female_group
female_group = female_group.loc[~is_outlier_females]
female_group

In [ ]:
## Checking for abs vlaue of z-scores that are > 3
is_outlier_males = np.abs(stats.zscore(male_group)) > 3
print(f"There are {is_outlier_males.sum()} outliers in the male group of out of {len(male_group)}.")

In [ ]:
## removing outliers from male_group
male_group = male_group.loc[~is_outlier_males]
male_group

### Checking for Normality

- According to the workflow on the LP, since both groups have n > 15, we can safely ignore the assumption of normality.

### Checking for Equal Variance

In [ ]:
result = stats.levene(male_group, female_group)
print(result)
print(result.pvalue<.05)

- According to the documentation for stats.levene, the null hypothesis for the test is that both groups have equal variance. Since our p-value is less than .05 we reject that null hypothesis and conclude that our groups do NOT have equal variance. 

- Since we did NOT meet the assumption of equal variance, we will run our stats.ttest_ind using equal_var=False. This will run a Welch's T-Test, which is designed to account for unequal variance. 

## Statistical Test 

In [ ]:
result = stats.ttest_ind(male_group, female_group, equal_var=False)
print(result)
result.pvalue < .05

## Final Conclusion

- Our Welch's T-Test return a p-value < .05 (it was actually p <.0001!) we reject the null hypothesis and support the alternative hypothesis that there is a significant difference in funded amounts for teams that included at least 1 female. 

- In order to know if they are funded significantly MORE or LESS, we look at the actual means of our final groups.

In [ ]:
print(f"The average funded_amount for male groups was {male_group.mean():.2f}")
print(f"The average funded_amount for female groups was {female_group.mean():.2f}")

- Male groups are funded at significantly higher amounts than female groups. 

## OPTIONAL - VIEWING THE BARPLOT WITHOUT OUTLIERS

In [ ]:
## concatenate the two dataframes for visualziation.
plot_df = pd.concat([male_df.loc[~is_outlier_males], 
                     female_df.loc[~is_outlier_females]], axis=0)
plot_df

In [ ]:
sns.barplot(data=plot_df, x='has_female',y='funded_amount')